In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
best_sweep = [16,0.2,0.000005,0.001,8]


#pretrained_bert = 'michiyasunaga/BioLinkBERT-base'
#pretrained_bert = 'dmis-lab/biobert-base-cased-v1.2'
#pretrained_bert = 'cimm-kzn/endr-bert'
#pretrained_bert = 'SpanBERT/spanbert-base-cased'
#pretrained_bert = 'emilyalsentzer/Bio_ClinicalBERT'
#pretrained_bert = 'dmis-lab/biobert-v1.1'
#pretrained_bert = 'bert-large-uncased'
pretrained_bert = 'allenai/scibert_scivocab_uncased'

#pretrained_bert = 'allenai/biomed_roberta_base' 
#pretrained_bert = 'microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract'


optim_batch_size = best_sweep[0]
optim_learning_rate = best_sweep[2]
optim_epochs = best_sweep[4]
optim_dropout = best_sweep[1]
weight_decay = best_sweep[3]

In [3]:
cropped_pretrained_bert = pretrained_bert.replace('/', '')
if pretrained_bert == 'bert-large-uncased':
    model_name = "PSY_CLASS_"+pretrained_bert
else:
    model_name = "PSY_CLASS_"+pretrained_bert.split('/')[1]

optim_model_name=pretrained_bert

#TRAINING FILE
use_huggingface = False
training_file = "PsyTAR_JC.csv"
# training_file = "**training**.csv"
column1_AEtext = "test" #"Example"
column2_AEflag = "label" #"AE Flag"
train_test_split_percent = 0.3
balance_multiplier = 1.0
#batch_size = 3
balance_me = True
#balance_me = False

#TEST FILE
test_file = "**test**.csv"
# test_file = "**test**.csv"
test_column1_AEtext = "test"
test_column2_AEflag = "label"

In [4]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
import numpy as np
import time
import datetime
import random
import os
import time
import pandas as pd
from torch.utils.data import TensorDataset, random_split
from tqdm.notebook import tqdm
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score,f1_score,precision_score


In [5]:
# use_huggingface = False
if use_huggingface == True:
    dataset = load_dataset("ade_corpus_v2","Ade_corpus_v2_classification")
else:
    df = pd.read_csv(training_file, encoding='unicode_escape')

In [6]:
if use_huggingface == True:
    #convert to pandas dataframe
    df = pd.DataFrame(dataset['train'])

In [7]:
num_AEs = df[column2_AEflag].value_counts()[1]
balance_number = int(num_AEs * balance_multiplier)

In [8]:
print(f"num_AEs = {num_AEs}\nbalance_number = {balance_number}\nbalance_multiplier = {balance_multiplier}")

num_AEs = 2168
balance_number = 2168
balance_multiplier = 1.0


In [9]:
if(balance_me == True):
    d_NoAE = df[df[column2_AEflag]==0]
    shuffle_d_NoAE = d_NoAE.sample(frac=1,random_state=42)[:balance_number]
    d_AE = df[df[column2_AEflag]==1]
    df = pd.concat([shuffle_d_NoAE,d_AE])

In [10]:
df[column2_AEflag].value_counts()

label
0    2168
1    2168
Name: count, dtype: int64

In [11]:
X = df[column1_AEtext]
y = df[column2_AEflag]

In [16]:
X_train,X_val,y_train, y_val = train_test_split(X,y, test_size=train_test_split_percent,stratify=y, random_state=42)

In [19]:
def prepare_data(model_name,batch_size):
    #tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True, use_fast=False)
    
    encoded_data_train = tokenizer.batch_encode_plus(X_train.values,
                                                    add_special_tokens=True,
                                                    return_attention_mask=True,
                                                    pad_to_max_length=True,
                                                    max_length=512,
                                                    return_tensors='pt')
    encoded_data_val = tokenizer.batch_encode_plus(X_val.values,
                                                    add_special_tokens=True,
                                                    return_attention_mask=True,
                                                    pad_to_max_length=True,
                                                    max_length=512,
                                                    return_tensors='pt')
    input_ids_train = encoded_data_train['input_ids']
    attention_masks_train = encoded_data_train['attention_mask']
    #labels = torch.tensor(dataset['AE Flag'].replace(label_dict).values)
    labels_train = torch.tensor(y_train.values)
    input_ids_val = encoded_data_val['input_ids']
    attention_masks_val = encoded_data_val['attention_mask']
    #labels = torch.tensor(dataset['AE Flag'].replace(label_dict).values)
    labels_val = torch.tensor(y_val.values)
    dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
    dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
    dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
    dataloader_validation = DataLoader(dataset_val, sampler = SequentialSampler(dataset_val), batch_size=batch_size)
    return dataloader_train,dataloader_validation

In [20]:
#checkLeul

def ret_model():
    
    configuration = AutoConfig.from_pretrained(pretrained_bert)
    configuration.hidden_dropout_prob = optim_dropout #check
    configuration.attention_probs_dropout_prob = optim_dropout #check
    configuration.num_labels = 2
    configuration.output_attentions = False
    configuration.output_hidden_states = False

    model = AutoModelForSequenceClassification.from_pretrained(pretrained_bert,config=configuration)
    return model

In [21]:
def f1_score_func(preds,labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [22]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [23]:
def train(batch_size,model_name,learning_rate,epochs,dropout): #checkleul, added dropout
    model=ret_model()
    dataloader_train,dataloader_validation =prepare_data(model_name,batch_size)
    optimizer = AdamW(model.parameters(),
                    lr = learning_rate, 
                    eps = 1e-8,weight_decay=weight_decay)
    total_steps = len(dataloader_train) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, 
                                                num_training_steps = total_steps)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    t0 = time.time()
    for epoch_i in range(0, epochs):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')
        total_train_loss = 0
        model.train()
        for step, batch in enumerate(dataloader_train):
            if step % 40 == 0 and not step == 0:
                print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(dataloader_train)))
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad()        
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels)
            loss = outputs[0]
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        avg_train_loss = total_train_loss / len(dataloader_train)            
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("")
        print("Running Validation...")
        model.eval()
        total_eval_loss = 0
        predictions, true_vals = [], []
        #nb_eval_steps = 0
        for batch in dataloader_validation:        
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)        
            with torch.no_grad():        
                outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask,
                                labels=b_labels)
            loss = outputs[0]
            logits = outputs[1]
            total_eval_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            predictions.append(logits)
            true_vals.append(label_ids)
        predictions = np.concatenate(predictions, axis=0)
        true_vals = np.concatenate(true_vals, axis=0)
        val_f1 = f1_score_func(predictions, true_vals)            
        print(f'Val_f1: {val_f1}')
        avg_val_loss = total_eval_loss / len(dataloader_validation)
        print(f'Validation loss: {avg_val_loss}')            
        if epoch_i+1 == epochs:
            torch.save(model.state_dict(), f'{cropped_pretrained_bert}_{epoch_i+1}.model')
    model.load_state_dict(torch.load(f'{cropped_pretrained_bert}_{epochs}.model', map_location=torch.device('cuda')))
    training_time = format_time(time.time() - t0)
    return model,training_time

In [24]:
def classify(text,tokenizer,device,model):
    encoded_input = tokenizer(text, return_tensors="pt",truncation=True,max_length=512,padding=True)
    encoded_input.to(device)
    output = model(**encoded_input)
    res = output.logits[0].detach().to('cpu').numpy()
    return np.argmax(res)

In [25]:
def classificationreporter(model):
    #tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)
    tokenizer = AutoTokenizer.from_pretrained(pretrained_bert, add_prefix_space=True, use_fast=False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    results=[]
    x=X_val.values
    y=y_val.values
    for text in x:
        results.append(classify(text,tokenizer,device,model))
    ground_truth=y
    conf = confusion_matrix(ground_truth, results)
    print('Confusion Matrix :')
    print(conf)
    print('Accuracy Score :',accuracy_score(ground_truth, results))
    print('Report : ')
    print(classification_report(ground_truth, results))
    return

In [26]:
model,training_time = train(optim_batch_size,optim_model_name,optim_learning_rate,optim_epochs,optim_dropout)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_le


======== Epoch 1 / 8 ========
Training...
  Batch    40  of    190.
  Batch    80  of    190.
  Batch   120  of    190.
  Batch   160  of    190.

  Average training loss: 0.54

Running Validation...
Val_f1: 0.7955368173070985
Validation loss: 0.43350476826109535

======== Epoch 2 / 8 ========
Training...
  Batch    40  of    190.
  Batch    80  of    190.
  Batch   120  of    190.
  Batch   160  of    190.

  Average training loss: 0.42

Running Validation...
Val_f1: 0.8285927820897587
Validation loss: 0.39657862657090515

======== Epoch 3 / 8 ========
Training...
  Batch    40  of    190.
  Batch    80  of    190.
  Batch   120  of    190.
  Batch   160  of    190.

  Average training loss: 0.37

Running Validation...
Val_f1: 0.8347126236632042
Validation loss: 0.3981866228689508

======== Epoch 4 / 8 ========
Training...
  Batch    40  of    190.
  Batch    80  of    190.
  Batch   120  of    190.
  Batch   160  of    190.

  Average training loss: 0.34

Running Validation...
Val_f

In [27]:
classificationreporter(model)

Confusion Matrix :
[[522 129]
 [ 82 568]]
Accuracy Score : 0.8378170637970792
Report : 
              precision    recall  f1-score   support

           0       0.86      0.80      0.83       651
           1       0.81      0.87      0.84       650

    accuracy                           0.84      1301
   macro avg       0.84      0.84      0.84      1301
weighted avg       0.84      0.84      0.84      1301



In [28]:
def reporter(model,x_data,y_data):
    #tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)
    tokenizer = AutoTokenizer.from_pretrained(pretrained_bert, add_prefix_space=True, use_fast=False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    results=[]
    for text in x_data.values:
        results.append(classify(text,tokenizer,device,model))
    ground_truth=y_data.values
    conf = confusion_matrix(ground_truth, results)
    recall = recall_score(ground_truth, results, pos_label=1)
    f1 = f1_score(ground_truth, results, pos_label=1)
    precision = precision_score(ground_truth, results, pos_label=1)
    return conf, recall,f1,precision

In [29]:
#first time
columns=["model_name","Training Time","precision","recall","f1_Score","True Positives","False Positives","True Negatives","False Negatives","Epochs","Learning Rate","Drop Out","Batch Size","Weight Decay"]
# df = pd.DataFrame(columns=columns)
# df.to_csv("results.csv",index=False)

In [30]:
def add_row(model):
  df=pd.read_csv("results.csv")
  x=X_val
  y=y_val
  conf,recall,f1,precision=reporter(model,x,y)
  data=[model_name,training_time,precision,recall,f1,conf[0][0],conf[0][1],conf[1][1],conf[1][0],optim_epochs,optim_learning_rate,optim_dropout,optim_batch_size,weight_decay]
  df.loc[len(df)] = data
  df.to_csv("results.csv",index=False)

In [31]:
add_row(model)